<a href="https://colab.research.google.com/github/AJAkil/AD340/blob/master/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2020-08-14 20:27:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 159.65.216.232, 104.248.60.43, 2604:a880:400:d1::895:1, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|159.65.216.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  3.30MB/s    in 7.6s    

2020-08-14 20:27:38 (3.28 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# Exploring the data-frames
df_books.head(5)

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [6]:
# Merging the data frames on the isbn column
combined_df = pd.merge(df_books, df_ratings, on='isbn')

combined_df.drop(['author'], axis=1, inplace=True)

In [7]:
combined_df.head()

,isbn,title,user,rating
0,0195153448,Classical Mythology,2,0.0
1,0002005018,Clara Callan,8,5.0
2,0002005018,Clara Callan,11400,0.0
3,0002005018,Clara Callan,11676,8.0
4,0002005018,Clara Callan,41385,0.0


In [8]:
len(combined_df)
# dropping any row that has a null value
combined_df.dropna(inplace=True)

Since we have so many data in the data set, we may need to filter out books based on the total number of ratings. So we groupby the dataframe by title and find out the total number of rating for each of the unique books.

In [9]:
# we group by the booktilte and take total rating in account
df_total_rating = (combined_df.groupby(by=['title'])['rating'].count().
                   reset_index().
                   rename(columns={'rating':'total_rating_count'})
                   [['title','total_rating_count']]
                   )

df_total_rating.head()

,title,total_rating_count
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [10]:
# We now combine this dataframe with our original one
combined_new_df = combined_df.merge(df_total_rating, left_on='title', right_on='title', how='inner')
combined_new_df.head()

,isbn,title,user,rating,total_rating_count
0,0195153448,Classical Mythology,2,0.0,2
1,0801319536,Classical Mythology,269782,7.0,2
2,0002005018,Clara Callan,8,5.0,14
3,0002005018,Clara Callan,11400,0.0,14
4,0002005018,Clara Callan,11676,8.0,14


In [11]:
df_total_rating['total_rating_count'].describe()

count    241090.000000
mean          4.277137
std          16.738045
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max        2502.000000
Name: total_rating_count, dtype: float64

So we see that the median book has been rated only once. So we need to explore the different quantiles to have an idea how to select the threshhold to filter rows based on total rating.

In [12]:
df_total_rating['total_rating_count'].quantile([.25,.5,.75])

0.25    1.0
0.50    1.0
0.75    3.0
Name: total_rating_count, dtype: float64

In [28]:
df_total_rating['total_rating_count'].quantile(np.arange(0.99,1,0.001))

241090

So it seems that only a small portion of the data distribution has a higher number of rating. Particularly, 1% books have a rating 50 or more. So we can keep the threshold to be 100 for the total rating. This would reduce the size of our data set.

In [14]:
threshhold = 100
combined_new_df = combined_new_df[combined_new_df['total_rating_count'] >= threshhold]
len(combined_new_df)

183799

In [15]:
combined_new_df.head()

,isbn,title,user,rating,total_rating_count
31,0399135782,The Kitchen God's Wife,8,0.0,311
32,0399135782,The Kitchen God's Wife,11676,9.0,311
33,0399135782,The Kitchen God's Wife,29526,9.0,311
34,0399135782,The Kitchen God's Wife,36836,0.0,311
35,0399135782,The Kitchen God's Wife,46398,9.0,311


# Filtering the dataset based on number of user rating
Since there are lot's of users with very low rating, so just like the previous book rating, we would like to filter the dataset based on the number of ratings given by the user. If a user has less than 200 rating, then that user will be ommitted from the dataset. This ensures another systematic way to filter the dataset.

In [50]:
# We convert the user unique ratings to a data frame
user_rating_count_df = combined_new_df['user'].value_counts().rename_axis('user').reset_index(name='counts')
user_rating_count_df.head(5)

,user,counts
0,11676,1173
1,35859,521
2,76352,433
3,16795,426
4,153662,396


In [52]:
# Then we merge the dataframe to a previous data frame
final_df = combined_new_df.merge(user_rating_count_df,how='left')

So we set the threshold to be 200 for gaining statistical signifance for our data set.

In [54]:
# We filter the final datafraame with a threshold of the total number of ratings
# to be greater or equal to 200
threshhold = 200
final_df = final_df[final_df['counts'] >= threshhold]
len(final_df)

14890

In [ ]:
user_threshhold = 200
combined_new_df = combined_new_df[combined_new_df['total_user_count'] >= user_threshhold]
len(combined_new_df)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()